In [32]:
%load_ext autoreload
%autoreload
%matplotlib inline

from news_crawler import NewsCrawler, select_urls
from IPython.display import display
from IPython.core.debugger import set_trace

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
crawler = NewsCrawler('news24','techcrunch','arstechnica','euronews','mirror','nbcnews','news.com.au','nextbigfuture','rt')

# 1. URL을 모은다

In [61]:
collect_results = crawler.collect(); display(*collect_results)

100.00% (64.01 seconds): Collecting URLs... rt                  

n_total     3473
n_unique    3473
dtype: int64

,pubs,actual_pub


news24           1202
techcrunch         15
arstechnica       211
euronews          937
mirror            206
nbcnews           198
news.com.au       267
nextbigfuture      28
rt                409
dtype: int64

# 2. 모은 URL중 이미 받은 것은 버린다

In [62]:
select_results = crawler.select(); display(*select_results)

100.00% (10.06 seconds): Selecting URLs... rt                  

n_total     3389
n_unique    3389
dtype: int64

,pubs,actual_pub


news24           1164
techcrunch         15
arstechnica       211
euronews          937
mirror            206
nbcnews           152
news.com.au       267
nextbigfuture      28
rt                409
dtype: int64

In [72]:
crawler.urls_selected['rt']

{'https://actualidad.rt.com/actualidad/325373-argentino-75-anos-cuenta-abusos-primera-vez',
 'https://actualidad.rt.com/actualidad/328813-grupo-pescadores-encontrar-argentina-gliptodonte',
 'https://actualidad.rt.com/actualidad/328902-polarizacion-politica-argentina-invadir-tinder',
 'https://actualidad.rt.com/actualidad/328905-entrevista-medioambientalista-guatemala-arriesgar-vida',
 'https://actualidad.rt.com/actualidad/328995-historia-gigantesco-complejo-favelas-brasil',
 'https://actualidad.rt.com/actualidad/329021-empresas-pena-calderon-otorgaron-perdon-impuestos-mexico',
 'https://actualidad.rt.com/actualidad/329222-argentino-detras-reggae-leo-messi-requena',
 'https://actualidad.rt.com/actualidad/329237-listos-venezuela-emite-primera-tarjeta-mastercard',
 'https://actualidad.rt.com/actualidad/329239-posibles-consecuencias-acuerdo-ecuador-fmi',
 'https://actualidad.rt.com/actualidad/329242-lenin-moreno-reafirma-paquete-medidas-protestas',
 'https://actualidad.rt.com/actualidad/32

# 3. 해당 URL의 뉴스를 다운로드한다

In [ ]:
download_results = crawler.download(); download_results

0.71% (17.37 seconds): Downloading... news24              

In [59]:
import hashlib
[hashlib.sha1(url.encode('utf-8')).hexdigest() for url in crawler.urls_final['financialpost']['trashed']]

['72cd5664fe0a9750948167b3a7b0cfed4be98426',
 '618814411e8f20a44d9b7b0f02b8e1d02de2b521',
 '7b9d9348c9c5c6e4cd1de29d3cbeb36a42f0687e',
 'b93060d7bd9d7b7923df6d818993e6f6d118e286',
 'd5011720567d4990422cab21bbab099521b782c4',
 '7af950abe69a58637a4208c9aac75695d6312ed4',
 'c414ac931f6cc1125214af3a5d3846f760fcd3e9',
 '422d6230a120e451e18c4922a7d8ecb218680445',
 '0e3bf8554b5ce03b258700efdb583d5f72de8371',
 '00616e93e781e8d4ea9e66e7cebb9fd555c7ba67',
 '5027d1d9aa085ac41d71bfb4792fb1d276a5eace']